In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [22]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [23]:
data = requests.get(standing_url)

In [24]:
soup = BeautifulSoup(data.text, 'html.parser')
# soup

In [25]:
standings_table = soup.select('table.stats_table')[0] # Get all team rankings table
standings_table

<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2024-202591_overall"> <caption>Premier League Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr> <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion." scope="col">Rk</th> <th aria-label="Squad" class="poptip sort_default_asc center" data-stat="team" scope="col">Squad</th> <th aria-label="Mat

In [26]:
links = standings_table.find_all('a')
links = [l.get('href') for l in links]
team_links = [l for l in links if '/squads/' in l] # Just get teams links
team_links = [f"https://fbref.com{l}" for l in team_links]
team_links

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/

In [27]:
# Test
team_url = team_links[0]
all_data = requests.get(team_url)
matches = pd.read_html(all_data.text, match='Scores & Fixtures')[0]
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,NaN,NaN,56.0,78146.0,Rúben Dias,4-3-3,4-2-3-1,Jarred Gillett,Match Report,Manchester City won on penalty kicks following...
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,0.8,1.0,52.0,39818.0,Kevin De Bruyne,3-2-4-1,4-2-3-1,Anthony Taylor,Match Report,NaN
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,3.3,0.3,75.0,53147.0,Kevin De Bruyne,3-2-4-1,5-4-1,Samuel Allison,Match Report,NaN
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,3.0,0.7,67.0,62469.0,Kevin De Bruyne,3-2-4-1,4-2-3-1,Michael Oliver,Match Report,NaN
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,2.1,1.0,54.0,52148.0,Kyle Walker,4-2-3-1,5-3-2,Darren Bond,Match Report,NaN


In [29]:
def get_links(data, cat):
    soup = BeautifulSoup(data.text, 'html.parser')
    links = soup.find_all('a')
    links = [l.get('href') for l in links]
    cat_links = [l for l in links if l and f'all_comps/{cat}/' in l]
    return cat_links

# rename second level columns
def rename_multiindex_col(df):
    columns = df.columns
    new_columns = pd.MultiIndex.from_tuples(
        [(cat, sub) if cat == columns[0][0]
         else (cat, cat + ' ' + sub) for cat, sub in columns]
    )
    df.columns = new_columns
    return df

In [30]:
# Shooting
shooting_links = get_links(all_data, 'shooting')
data = requests.get(f"https://fbref.com{shooting_links[0]}")
shooting = pd.read_html(data.text, match='Shooting')[0]
shooting = rename_multiindex_col(shooting)
# Drop the first index level
shooting.columns = shooting.columns.droplevel(0)
shooting.drop(columns=['Time', 'Comp', 'Round', 'Venue', 'Day', 'Opponent', 'Result', 'GF', 'GA', 'Standard SoT%', 'Standard G/Sh', 'Standard G/SoT',
                       'Expected npxG/Sh', 'Expected G-xG', 'Expected np:G-xG', 'Unnamed: 25_level_0 Match Report'], inplace=True)
shooting.head()

,Date,Standard Gls,Standard Sh,Standard SoT,Standard Dist,Standard FK,Standard PK,Standard PKatt,Expected xG,Expected npxG
0,2024-08-10,1,9,1,NaN,NaN,0,0,NaN,NaN
1,2024-08-18,2,11,5,19.1,0.0,0,0,0.8,0.8
2,2024-08-24,4,13,4,17.8,1.0,1,1,3.3,2.6
3,2024-08-31,3,23,8,15.0,1.0,0,0,3.0,3.0
4,2024-09-14,2,18,7,17.2,0.0,0,0,2.1,2.1


In [31]:
# Goalkeeping
gk_links = get_links(all_data, 'keeper')
data = requests.get(f"https://fbref.com{gk_links[0]}")
gk = pd.read_html(data.text, match='Goalkeeping')[0]
gk.drop(columns=['Penalty Kicks'], inplace=True)

columns = gk.columns
columns = pd.MultiIndex.from_tuples(
    [(cat, 'lauched att' if (cat == 'Launched' and sub == 'Att') else
      'goal kick att' if (cat == 'Goal Kicks' and sub == 'Att') else
      'pass avglen' if (cat == 'Passes' and sub == 'AvgLen') else
             'goal kick avglen' if (cat == 'Goal Kicks' and sub == 'AvgLen') else sub)
     for cat, sub in columns]
)
gk.columns = columns
gk.columns = gk.columns.droplevel(0)
gk.drop(columns=['Comp', 'Time', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Save%', 'PSxG+/-', 'Cmp%', 'Launch%', 'Stp%', 'Match Report'], inplace=True)
gk.rename(columns={'Cmp' : 'launched cmp', 'Att (GK)' : 'passes att (gk)', 'Opp' : 'crosses faced', 'Stp' : 'crosses stp', 'AvgDist' : 'sweeper avgdist'}, inplace=True)
gk.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_8116\3233747620.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  gk.drop(columns=['Penalty Kicks'], inplace=True)


,Date,SoTA,Saves,CS,PSxG,launched cmp,lauched att,passes att (gk),Thr,pass avglen,goal kick att,goal kick avglen,crosses faced,crosses stp,#OPA,sweeper avgdist
0,2024-08-10,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-18,3,3,1,0.7,4.0,9.0,37.0,5.0,29.5,5.0,28.6,12.0,1.0,0.0,11.0
2,2024-08-24,1,0,0,0.3,1.0,1.0,24.0,1.0,24.1,1.0,8.0,5.0,0.0,1.0,19.5
3,2024-08-31,2,2,0,0.2,3.0,7.0,33.0,2.0,27.4,6.0,21.2,10.0,0.0,2.0,24.0
4,2024-09-14,5,4,0,2.1,6.0,15.0,41.0,2.0,31.7,3.0,67.7,9.0,0.0,6.0,16.3


In [32]:
# Passing
passing_links = get_links(all_data, 'passing')
data = requests.get(f"https://fbref.com{passing_links[0]}")
passing = pd.read_html(data.text, match='Passing')[0]
passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)
passing.columns = passing.columns.droplevel(0)
passing.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Cmp%', 'Match Report'], inplace=True)
passing.rename(columns={'Cmp' : 'passes cmp', 'Att' : 'passes att', 'TotDist' : 'total passing dist', 'PrgDist' : 'progressive passing dist'}, inplace=True)
passing.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_8116\470473815.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)


,Date,passes cmp,passes att,total passing dist,progressive passing dist,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP
0,2024-08-10,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-18,501.0,570.0,8217.0,2491.0,1,0.6,0.9,8.0,29.0,6.0,0.0,32.0
2,2024-08-24,726.0,796.0,11491.0,2774.0,2,2.3,1.9,11.0,61.0,18.0,4.0,60.0
3,2024-08-31,691.0,762.0,11460.0,3117.0,3,2.7,2.7,20.0,67.0,19.0,1.0,77.0
4,2024-09-14,507.0,581.0,8423.0,2436.0,1,2.0,1.5,16.0,35.0,19.0,0.0,55.0


In [33]:
# Pass Types
pass_types_links = get_links(all_data, 'passing_types')

data = requests.get(f"https://fbref.com{pass_types_links[0]}")
pass_types = pd.read_html(data.text, match='Pass Types')[0]
pass_types.columns = pass_types.columns.droplevel(0)
pass_types.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Att', 'Match Report',
                         'Cmp'], inplace=True)
pass_types.rename(columns={'In' : 'inswing conrner kicks', 'Out' : 'outswing corner kicks', 'Str' : 'straight corner kicks', 'Off' : 'passes offside',
                           'Blocks' : 'passes blocked'}, inplace=True)
pass_types.head()


,Date,Live,Dead,FK,TB,Sw,Crs,TI,CK,inswing conrner kicks,outswing corner kicks,straight corner kicks,passes offside,passes blocked
0,2024-08-10,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-18,528.0,41.0,14.0,2.0,1.0,11,18.0,3.0,2.0,0.0,0.0,1.0,7.0
2,2024-08-24,747.0,46.0,13.0,2.0,2.0,17,20.0,10.0,3.0,1.0,0.0,3.0,13.0
3,2024-08-31,722.0,39.0,9.0,4.0,2.0,22,11.0,11.0,2.0,3.0,1.0,1.0,9.0
4,2024-09-14,540.0,40.0,4.0,1.0,2.0,15,19.0,12.0,4.0,0.0,0.0,1.0,12.0


In [34]:
# Goal and Shot Creation 
gs_links = get_links(all_data, 'gca')
data = requests.get(f"https://fbref.com{gs_links[0]}")
gs = pd.read_html(data.text, match='Goal and Shot Creation')[0]

gs = rename_multiindex_col(gs)

gs.columns = gs.columns.droplevel(0)
gs.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'SCA Types SCA', 'GCA Types GCA', 'Unnamed: 24_level_0 Match Report'], inplace=True)
gs.head()

,Date,SCA Types PassLive,SCA Types PassDead,SCA Types TO,SCA Types Sh,SCA Types Fld,SCA Types Def,GCA Types PassLive,GCA Types PassDead,GCA Types TO,GCA Types Sh,GCA Types Fld,GCA Types Def
0,2024-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-18,16.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,2024-08-24,22.0,2.0,1.0,0.0,2.0,1.0,5.0,0.0,1.0,0.0,1.0,1.0
3,2024-08-31,31.0,5.0,2.0,3.0,1.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0
4,2024-09-14,29.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# defensive actions
def_actions_links = get_links(all_data, 'defense')
data = requests.get(f"https://fbref.com{def_actions_links[0]}")
def_actions = pd.read_html(data.text, match='Defensive Actions')[0]
def_actions = rename_multiindex_col(def_actions)
def_actions.columns = def_actions.columns.droplevel(0)
def_actions.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Unnamed: 26_level_0 Match Report'], inplace=True)
def_actions.rename(columns={'Unnamed: 22_level_0 Int' : 'Int', 'Unnamed: 23_level_0 Tkl+Int' : 'Tkl + Int', 'Unnamed: 24_level_0 Clr' : 'Clr',
                            'Unnamed: 25_level_0 Err' : 'Err'}, inplace=True)

def_actions.head()

,Date,Tackles Tkl,Tackles TklW,Tackles Def 3rd,Tackles Mid 3rd,Tackles Att 3rd,Challenges Tkl,Challenges Att,Challenges Tkl%,Challenges Lost,Blocks Blocks,Blocks Sh,Blocks Pass,Int,Tkl + Int,Clr,Err
0,2024-08-10,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,0,NaN,NaN
1,2024-08-18,16.0,9,10.0,4.0,2.0,7.0,11.0,63.6,4.0,6.0,4.0,2.0,10,26,18.0,0.0
2,2024-08-24,14.0,6,5.0,4.0,5.0,6.0,8.0,75.0,2.0,8.0,0.0,8.0,6,20,9.0,0.0
3,2024-08-31,15.0,12,8.0,4.0,3.0,5.0,15.0,33.3,10.0,14.0,3.0,11.0,7,22,19.0,1.0
4,2024-09-14,12.0,5,5.0,3.0,4.0,6.0,9.0,66.7,3.0,8.0,1.0,7.0,5,17,13.0,3.0


In [36]:
# Possession
possession_links = get_links(all_data, 'possession')
data = requests.get(f"https://fbref.com{possession_links[0]}")
possession = pd.read_html(data.text, match='Possession')[0]
possession = rename_multiindex_col(possession)
possession.columns = possession.columns.droplevel(0)
possession.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Unnamed: 33_level_0 Match Report', 'Take-Ons Succ%',
                         'Take-Ons Tkld%'], inplace=True)
possession.head()

,Date,Poss,Touches Touches,Touches Def Pen,Touches Def 3rd,Touches Mid 3rd,Touches Att 3rd,Touches Att Pen,Touches Live,Take-Ons Att,...,Carries Carries,Carries TotDist,Carries PrgDist,Carries PrgC,Carries 1/3,Carries CPA,Carries Mis,Carries Dis,Receiving Rec,Receiving PrgR
0,2024-08-10,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-18,52.0,664.0,76.0,238.0,261.0,178.0,24.0,664.0,30.0,...,385.0,2736.0,1620.0,41.0,31.0,13.0,8.0,12.0,496.0,32.0
2,2024-08-24,75.0,884.0,33.0,126.0,363.0,401.0,47.0,883.0,20.0,...,501.0,2654.0,1510.0,27.0,27.0,11.0,16.0,10.0,718.0,60.0
3,2024-08-31,67.0,857.0,74.0,195.0,362.0,303.0,57.0,857.0,19.0,...,461.0,2857.0,1695.0,39.0,20.0,16.0,7.0,8.0,681.0,77.0
4,2024-09-14,54.0,671.0,47.0,151.0,195.0,329.0,51.0,671.0,14.0,...,341.0,2101.0,1111.0,25.0,22.0,14.0,16.0,12.0,495.0,54.0


In [37]:
# Miscellaneous Stats
misc_stats_links = get_links(all_data, 'misc')
data = requests.get(f"https://fbref.com{misc_stats_links[0]}")
misc_stats = pd.read_html(data.text, match='Miscellaneous Stats')[0]
misc_stats = rename_multiindex_col(misc_stats)
misc_stats.columns = misc_stats.columns.droplevel(0)
misc_stats.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Aerial Duels Won%', 'Unnamed: 26_level_0 Match Report'], inplace=True)
misc_stats.head()

,Date,Performance CrdY,Performance CrdR,Performance 2CrdY,Performance Fls,Performance Fld,Performance Off,Performance Crs,Performance Int,Performance TklW,Performance PKwon,Performance PKcon,Performance OG,Performance Recov,Aerial Duels Won,Aerial Duels Lost
0,2024-08-10,2,0,0,10,9,2,12,6,6,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-18,1,0,0,9,12,1,11,10,9,0.0,0.0,0.0,41.0,5.0,3.0
2,2024-08-24,2,0,0,4,15,3,17,6,6,1.0,0.0,0.0,32.0,5.0,7.0
3,2024-08-31,2,0,0,3,10,1,22,7,12,0.0,0.0,1.0,43.0,7.0,9.0
4,2024-09-14,3,0,0,9,3,1,15,5,5,0.0,0.0,0.0,38.0,6.0,11.0


In [38]:
team_data = matches.merge(shooting, on='Date')
team_data = team_data.merge(gk, on='Date')
team_data = team_data.merge(passing, on='Date')
team_data = team_data.merge(pass_types, on='Date')
team_data = team_data.merge(gs, on='Date')
team_data = team_data.merge(def_actions, on='Date')
team_data = team_data.merge(possession, on='Date')
team_data = team_data.merge(misc_stats, on='Date')

In [39]:
team_data.columns = team_data.columns.str.lower()
team_data = team_data[team_data['comp'] == 'Premier League']
team_data['round'] = team_data['round'].str.extract(r'(\d+)')
team_data.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,performance off,performance crs,performance int,performance tklw,performance pkwon,performance pkcon,performance og,performance recov,aerial duels won,aerial duels lost
1,2024-08-18,16:30,Premier League,1,Sun,Away,W,2,0,Chelsea,...,1,11,10,9,0.0,0.0,0.0,41.0,5.0,3.0
2,2024-08-24,15:00,Premier League,2,Sat,Home,W,4,1,Ipswich Town,...,3,17,6,6,1.0,0.0,0.0,32.0,5.0,7.0
3,2024-08-31,17:30,Premier League,3,Sat,Away,W,3,1,West Ham,...,1,22,7,12,0.0,0.0,1.0,43.0,7.0,9.0
4,2024-09-14,15:00,Premier League,4,Sat,Home,W,2,1,Brentford,...,1,15,5,5,0.0,0.0,0.0,38.0,6.0,11.0
6,2024-09-22,16:30,Premier League,5,Sun,Home,D,2,2,Arsenal,...,0,24,3,5,0.0,0.0,0.0,39.0,16.0,11.0


In [40]:
# shape 3 cai nay khac nhau vi matches bao gom tat ca cac tran dau, shooting chi bao gom cac tran da da
matches.shape, shooting.shape, team_data.shape


((49, 20), (15, 10), (9, 133))

In [2]:
'''Begin'''

'Begin'

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def save_progress(year, team_name):
    with open('progress.txt', 'w') as f:
        f.write(f"{year},{team_name}")

def load_progress():
    try:
        with open('progress.txt', 'r') as f:
            progress = f.read().strip().split(',')
            return int(progress[0]), progress[1]  # Return last year, last team
    except FileNotFoundError:
        return None, None

def save_data(all_matches):
    df = pd.concat(all_matches)
    if not os.path.exists('matches.csv'):
        df.to_csv('matches.csv', index=False)
    else:
        df.to_csv('matches.csv', mode='a', header=False, index=False)

def get_links(data, cat):
    soup = BeautifulSoup(data.text, 'html.parser')
    links = soup.find_all('a')
    links = [l.get('href') for l in links]
    cat_links = [l for l in links if l and f'all_comps/{cat}/' in l]
    return cat_links

# rename second level columns
def rename_multiindex_col(df):
    columns = df.columns
    new_columns = pd.MultiIndex.from_tuples(
        [(cat, sub) if cat == columns[0][0]
         else (cat, cat + ' ' + sub) for cat, sub in columns]
    )
    df.columns = new_columns
    return df

# shooting
def process_shooting(data):
    shooting_links = get_links(data, 'shooting')
    shooting_data = requests.get(f"https://fbref.com{shooting_links[0]}")
    shooting = pd.read_html(shooting_data.text, match='Shooting')[0]
    shooting = rename_multiindex_col(shooting)
    shooting.columns = shooting.columns.droplevel(0)
    shooting.drop(columns=['Time', 'Comp', 'Round', 'Venue', 'Day', 'Opponent', 'Result', 'GF', 'GA', 'Standard SoT%', 'Standard G/Sh', 'Standard G/SoT',
                        'Expected npxG/Sh', 'Expected G-xG', 'Expected np:G-xG', 'Unnamed: 25_level_0 Match Report'], inplace=True)
    return shooting

# goalkeeping
def process_gk(data):
    gk_links = get_links(data, 'keeper')
    gk_data = requests.get(f"https://fbref.com{gk_links[0]}")
    gk = pd.read_html(gk_data.text, match='Goalkeeping')[0]
    gk.drop(columns=['Penalty Kicks'], inplace=True)
    columns = gk.columns
    columns = pd.MultiIndex.from_tuples(
        [(cat, 'lauched att' if (cat == 'Launched' and sub == 'Att') else
        'goal kick att' if (cat == 'Goal Kicks' and sub == 'Att') else
        'pass avglen' if (cat == 'Passes' and sub == 'AvgLen') else
                'goal kick avglen' if (cat == 'Goal Kicks' and sub == 'AvgLen') else sub)
        for cat, sub in columns]
    )
    gk.columns = columns
    gk.columns = gk.columns.droplevel(0)
    gk.drop(columns=['Comp', 'Time', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Save%', 'PSxG+/-', 'Cmp%', 'Launch%', 'Stp%', 'Match Report'], inplace=True)
    gk.rename(columns={'Cmp' : 'launched cmp', 'Att (GK)' : 'passes att (gk)', 'Opp' : 'crosses faced', 'Stp' : 'crosses stp', 'AvgDist' : 'sweeper avgdist'}, inplace=True)
    return gk

# passing
def process_passing(data):
    passing_links = get_links(data, 'passing')
    passing_data = requests.get(f"https://fbref.com{passing_links[0]}")
    passing = pd.read_html(passing_data.text, match='Passing')[0]
    passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)
    passing.columns = passing.columns.droplevel(0)
    passing.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Cmp%', 'Match Report'], inplace=True)
    passing.rename(columns={'Cmp' : 'passes cmp', 'Att' : 'passes att', 'TotDist' : 'total passing dist', 'PrgDist' : 'progressive passing dist'}, inplace=True)
    return passing

# pass types
def process_pass_types(data):
    pass_types_links = get_links(data, 'passing_types')
    pass_types_data = requests.get(f"https://fbref.com{pass_types_links[0]}")
    pass_types = pd.read_html(pass_types_data.text, match='Pass Types')[0]
    pass_types.columns = pass_types.columns.droplevel(0)
    pass_types.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Att', 'Match Report',
                            'Cmp'], inplace=True)
    pass_types.rename(columns={'In' : 'inswing conrner kicks', 'Out' : 'outswing corner kicks', 'Str' : 'straight corner kicks', 'Off' : 'passes offside',
                                    'Blocks' : 'passes blocked'}, inplace=True)
    return pass_types

# Goal and Shot Creation
def process_gs(data):
    gs_links = get_links(data, 'gca')
    gs_data = requests.get(f"https://fbref.com{gs_links[0]}")
    gs = pd.read_html(gs_data.text, match='Goal and Shot Creation')[0]
    gs = rename_multiindex_col(gs)
    gs.columns = gs.columns.droplevel(0)
    gs.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'SCA Types SCA', 
                     'GCA Types GCA', 'Unnamed: 24_level_0 Match Report'], inplace=True)
    return gs

# defensive actions
def process_def_actions(data):
    def_actions_links = get_links(data, 'defense')
    def_actions_data = requests.get(f"https://fbref.com{def_actions_links[0]}")
    def_actions = pd.read_html(def_actions_data.text, match='Defensive Actions')[0]
    def_actions = rename_multiindex_col(def_actions)
    def_actions.columns = def_actions.columns.droplevel(0)
    def_actions.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Unnamed: 26_level_0 Match Report'], inplace=True)
    def_actions.rename(columns={'Unnamed: 22_level_0 Int' : 'Int', 'Unnamed: 23_level_0 Tkl+Int' : 'Tkl + Int', 'Unnamed: 24_level_0 Clr' : 'Clr',
                                'Unnamed: 25_level_0 Err' : 'Err'}, inplace=True)
    return def_actions

# Possession
def process_possession(data):
    possession_links = get_links(data, 'possession')
    possession_data = requests.get(f"https://fbref.com{possession_links[0]}")
    possession = pd.read_html(possession_data.text, match='Possession')[0]
    possession = rename_multiindex_col(possession)
    possession.columns = possession.columns.droplevel(0)
    possession.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Unnamed: 33_level_0 Match Report', 'Take-Ons Succ%',
                                    'Take-Ons Tkld%'], inplace=True)
    return possession

# Miscellaneous Stats
def process_misc_stats(data):
    misc_stats_links = get_links(data, 'misc')
    misc_data = requests.get(f"https://fbref.com{misc_stats_links[0]}")
    misc_stats = pd.read_html(misc_data.text, match='Miscellaneous Stats')[0]
    misc_stats = rename_multiindex_col(misc_stats)
    misc_stats.columns = misc_stats.columns.droplevel(0)
    misc_stats.drop(columns=['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Aerial Duels Won%', 
                             'Unnamed: 26_level_0 Match Report'], inplace=True)
    return misc_stats



years = list(range(2024, 2016, -1))
# all_matches = [] 
# standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

# Load progress nếu có
last_year, last_team = load_progress()

for year in years:
    if last_year and year > last_year: 
        continue
    try:
        last_year_url = f"https://fbref.com/en/comps/9/{year}-{year+1}/{year}-{year+1}-Premier-League-Stats"
        data = requests.get(last_year_url)
        data.raise_for_status()  # Check for rate limit
    except requests.exceptions.HTTPError as e:
        print(f"Request failed: {e}")
        break

    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l.get('href') for l in links]
    team_links = [l for l in links if '/squads/' in l]
    team_links = [f"https://fbref.com{l}" for l in team_links]
    
    previous_season = soup.select('a.prev')[0].get('href')
    standing_url = f"https://fbref.com{previous_season}"

    for team_link in team_links:
        team_name = team_link.split('/')[-1].replace('-Stats', '').replace('-', ' ')
        
        if last_team and team_name != last_team:
            continue
        last_team = None

        save_progress(year, team_name)  

        try:
            data = requests.get(team_link)
            # data.raise_for_status()

            matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
            shooting = process_shooting(data)
            gk = process_gk(data)
            passing = process_passing(data)
            pass_types = process_pass_types(data)
            gs = process_gs(data)
            def_actions = process_def_actions(data)
            possession = process_possession(data)
            misc_stats = process_misc_stats(data)

            team_data = matches.merge(shooting, on='Date')
            team_data = team_data.merge(gk, on='Date')
            team_data = team_data.merge(passing, on='Date')
            team_data = team_data.merge(pass_types, on='Date')
            team_data = team_data.merge(gs, on='Date')
            team_data = team_data.merge(def_actions, on='Date')
            team_data = team_data.merge(possession, on='Date')
            team_data = team_data.merge(misc_stats, on='Date')

            # Lọc ra các trận chỉ thuộc Premier League
            team_data = team_data[team_data['Comp'] == 'Premier League']
            team_data['Season'] = str(year) + '-' + str(year + 1)
            team_data['Team'] = team_name
            # all_matches.append(team_data)
            print(f"Data for {team_name} in {year} season has been scraped.")
            
            # Lưu dữ liệu vào CSV sau khi lấy xong dữ liệu của một đội
            save_data([team_data])
        
        except ValueError:
            continue
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                print("rate limit exceeded")
                break
        time.sleep(5)  


if os.path.exists('progress.txt'):
    os.remove('progress.txt') 


C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:61: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  gk.drop(columns=['Penalty Kicks'], inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)


Data for Arsenal in 2024 season has been scraped.


C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:61: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  gk.drop(columns=['Penalty Kicks'], inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)


Data for Aston Villa in 2024 season has been scraped.


C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:61: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  gk.drop(columns=['Penalty Kicks'], inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)


...

Data for Brighton and Hove Albion in 2022 season has been scraped.


C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:61: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  gk.drop(columns=['Penalty Kicks'], inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)


Data for Aston Villa in 2022 season has been scraped.


C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:61: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  gk.drop(columns=['Penalty Kicks'], inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_8668\232733087.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  passing.drop(columns=['Short', 'Medium', 'Long'], inplace=True)


ConnectionError: HTTPSConnectionPool(host='fbref.com', port=443): Max retries exceeded with url: /en/squads/361ca564/2022-2023/matchlogs/all_comps/defense/Tottenham-Hotspur-Match-Logs-All-Competitions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D2C91C6AF0>: Failed to establish a new connection: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions'))

In [3]:
import pandas as pd

with open('matches.csv', 'r') as f:
    df = pd.read_csv(f)
    df['Season'] = df['Season'].astype(int).apply(lambda x: f"{x}-{x+1}")
    df.to_csv('new_matches.csv', index=False) 

In [6]:
import pandas as pd

df = pd.read_csv('new_matches.csv')
for idx, formation in enumerate(df['Formation']):
    if "/" in formation:
        fi, se, thi = formation.split("/")
        formation = fi + "-" + se + "-" + str(10 - int(fi) - int(se))
        # print(formation)
    if "0" in formation:
        formation = formation.replace("-0", "")
        # print(formation)
    df.at[idx, 'Formation'] = formation

for idx, formation in enumerate(df['Opp Formation']):
    if "/" in formation:
        fi, se, thi = formation.split("/")
        formation = fi + "-" + se + "-" + str(10 - int(fi) - int(se))
    if "0" in formation:
        formation = formation.replace("-0", "")
    df.at[idx, 'Opp Formation'] = formation
df.to_csv('final.csv', index=False)

In [49]:
import pandas as pd
df = pd.read_csv('new_matches.csv')
df.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation',
       'Opp Formation', 'Referee', 'Match Report', 'Notes', 'Sh', 'SoT',
       'Dist', 'FK', 'PK', 'PKatt', 'Season', 'Team'],
      dtype='object')